In [1]:
import pandas as pd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
load_dotenv()
import os
from datetime import datetime, timedelta
from tqdm import tqdm
from processor.processor import Processor as p
from database.adatabase import ADatabase

In [2]:
market = ADatabase("market")
market.connect()
index = market.retrieve("sp500")
market.disconnect()

In [3]:
start = datetime(2024,1,1)
end = datetime.now()

In [5]:
prices = []
date = start
market.connect()
for ticker in tqdm(index["ticker"]):
    try:
        price = market.query("prices",{"ticker":ticker})
        price = p.column_date_processing(price)
        price.sort_values("date",inplace=True)
        prices.append(price)
    except Exception as e:
        print(str(e))
market.disconnect()

 12%|█████████████                                                                                                 | 60/503 [00:17<02:10,  3.39it/s]

'date'


 15%|████████████████▍                                                                                             | 75/503 [00:22<02:32,  2.81it/s]

'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [02:29<00:00,  3.37it/s]

'date'


In [6]:
market_prices = pd.concat(prices)

In [7]:
def calculate_indicators(price, timeframe):
    """Calculate indicators for a single ticker."""
    price = price.sort_values("date")
    price["sma"] = price["adjclose"].rolling(timeframe).mean()
    price["ema"] = price["adjclose"].ewm(span=timeframe, adjust=False).mean()

    delta = price["adjclose"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(timeframe).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(timeframe).mean()
    rs = gain / loss
    price["rsi"] = 100 - (100 / (1 + rs))

    price["std"] = price["adjclose"].rolling(timeframe).std()
    price["bollinger_upper"] = price["sma"] + 2 * price["std"]
    price["bollinger_lower"] = price["sma"] - 2 * price["std"]

    price["pct_change"] = price["adjclose"].pct_change(periods=timeframe)
    price["momentum"] = price["adjclose"] - price["adjclose"].shift(timeframe)

    price["volatility"] = price["adjclose"].rolling(timeframe).std()
    price["coev"] = price["adjclose"].rolling(timeframe).std() / price["adjclose"].rolling(timeframe).mean()

    return price.dropna()

In [45]:
prices = []
timeframe = 100
for ticker in tqdm(market_prices["ticker"].unique()):
    try:
        price = market_prices[market_prices["ticker"] == ticker]
        price = calculate_indicators(price, timeframe)
        prices.append(price)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Combine and sort all price data
sim = pd.concat(prices).dropna().sort_values("date")
sim["year"] = sim["date"].dt.year
sim["week"] = [x.week for x in sim["date"]]
sim["sell_price"] = sim["adjclose"]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [10:38<00:00,  1.28s/it]


In [46]:
signals = ["sma"
           ,"ema","rsi","volatility","bollinger_upper","bollinger_lower","momentum","pct_change","coev"
          ]
ascendings = [True,False]

In [47]:
analysis = []
for signal in signals:
    for ascending in ascendings:
        opportunities = sim[sim["year"]>2012].groupby(["year","week","ticker"]).agg({"date":"last","adjclose":"first","sell_price":"last",signal:"first"}).reset_index()
        trades = opportunities.sort_values(signal,ascending=ascending).groupby(["year","week"]).first().reset_index()
        trades.sort_values("date",inplace=True)
        trades["return"] = trades["sell_price"] / trades["adjclose"]
        trades["pnl"] = trades["return"].cumprod()
        trades["downside"] = trades["pnl"].rolling(100).max() - trades["pnl"]
        analysis.append({
            "signal":signal,
            "ascending":ascending,
            "pnl":trades["pnl"].iloc[-1],
            "downside":trades["downside"].max(),
            "volatility": trades["return"].std()
        })

In [48]:
a = pd.DataFrame(analysis)
a["risk"] = a["downside"] * a["volatility"]

In [49]:
a.sort_values("risk",ascending=True).head(50)

,signal,ascending,pnl,downside,volatility,risk
7,volatility,False,9.312654,1.462237,0.044274,0.064739
11,bollinger_lower,False,9.140506,1.513883,0.043166,0.065349
1,sma,False,9.096748,1.506635,0.043405,0.065395
3,ema,False,9.373537,1.552478,0.043400,0.067378
9,bollinger_upper,False,9.472506,1.568870,0.043447,0.068162
13,momentum,False,4.458965,1.902260,0.037940,0.072172
16,coev,True,6.693704,2.730743,0.031376,0.085679
12,momentum,True,5.778976,4.084054,0.055333,0.225984
5,rsi,False,2.499461,8.791324,0.066969,0.588748
6,volatility,True,38.340133,9.624768,0.061195,0.588984
